In [1]:
import polars as pl
import os
from util import pdf_to_text, best_fuzzy_match

In [2]:
ms_cases = [c.replace(".pdf", "") for c in os.listdir("Cases/IncompleteOpinions/")]
df = pl.read_csv("./Cases/IncompleteOpinions.csv")
df.head()

Title,DATE
str,str
"""Rogers v. Koon…","""1992/04/02"""
"""Kienitz v. Sco…","""2014/09/15"""
"""Reyher v. Chil…","""1976/04/05"""
"""TCA Television…","""2016/10/11"""
"""Society of the…","""2010/12/03"""


In [3]:
df = df.with_columns(
    pl.col("Title").map_elements(lambda x: best_fuzzy_match(x, choices=ms_cases)[0]+".pdf").alias("Matched"),
    pl.col("Title").map_elements(lambda x: best_fuzzy_match(x, choices=ms_cases)[1]).alias("MatchedScore")
).sort("MatchedScore")

In [5]:
df = df.with_columns(
    pl.col("Matched").map_elements(lambda x: pdf_to_text("./Cases/IncompleteOpinions/" + x)).alias("Opinion")
)

In [6]:
df.write_csv("./Cases/Incomplete_Opinion.csv")